# In Class Lab 1

## Part A:  The Local Standard of Rest
Proper motion of Sgr A* from Reid & Brunthaler 2004
$\mu = 6.379$ mas/yr 

Peculiar motion of the sun, $v_\odot$ = 12.24 km/s  (Schonrich 2010)


$v_{tan} = 4.74 \frac{\mu}{\rm mas/yr} \frac{R_o}{\rm kpc} = V_{LSR} + v_\odot$


### a)

Create a function called VLSR to compute the local standard of res (V$_{LSR}$).

The function should take as input: the solar radius (R$_o$), the proper motion (mu)
and the peculiar motion of the sun in the $v_\odot$ direction.

Compute V$_{LSR}$ using three different values R$_o$: 
1. Water Maser Distance for the Sun :  R$_o$ = 8.34 kpc   (Reid 2014 ApJ 783) 
2. GRAVITY Collaboration Distance for the Sun:  R$_o$ = 8.178 kpc   (Abuter+2019 A&A 625)
3. Value for Distance to Sun listed in Sparke & Gallagher : R$_o$ = 7.9 kpc 

In [1]:
import numpy as np
import astropy.units as u

In [2]:
def VLSR(R_0, mu, v_sun):
    """
    Args:
        R_0 (Quantity): Solar radius (galactocentric distance)
        mu (Quantity): proper motion
        v_sun (Quantity): peculiar motion of the Sun in the v_sun direction
        
    Function to compute and return the local standard of rest velocity (km/s)
    """
    
    v_tan = (4.74 * mu.to(u.mas/u.year).value * R_0.to(u.kpc).value) * u.km/u.s
    return v_tan - v_sun.to(u.km/u.s)

In [3]:
mu = 6.379 * u.mas / u.year
v_sun = 12.24 * u.km/u.s
R_0 = np.array([8.34, 8.178, 7.9]) * u.kpc

In [4]:
V_LSR = VLSR(R_0, mu, v_sun)
display(V_LSR)

# normalize on book value of R_0:
print(f'Relative velocities: {np.around(V_LSR / V_LSR[2], 3)}')

<Quantity [239.9320764 , 235.03376988, 226.628034  ] km / s>

Relative velocities: [1.059 1.037 1.   ]


### b)

compute the orbital period of the sun using R$_o$ from the GRAVITY Collaboration (assume circular orbit)

Note that 1 km/s $\sim$ 1kpc/Gyr

In [5]:
circumference = 2 * np.pi * R_0
period = circumference / V_LSR
maser_period, gravity_period, book_period = period.to(u.Gyr)
print(f'Orbital period of the sun using GRAVITY R_0 is {gravity_period:.3f}')

Orbital period of the sun using GRAVITY R_0 is 0.214 Gyr


### c)

Compute the number of rotations about the GC over the age of the universe (13.8 Gyr)

In [6]:
cosmo_age = 13.8 * u.Gyr
rotations = cosmo_age / gravity_period
print(f'Rotations so far: {rotations:.1f}')

Rotations so far: 64.6


## Part B  Dark Matter Density Profiles

### a)
Try out Fitting Rotation Curves 
[here](http://wittman.physics.ucdavis.edu/Animations/RotationCurve/GalacticRotation.html)


### b)

In the Isothermal Sphere model, what is the mass enclosed within the solar radius (R$_o$) in units of $10^{10}$ M$_\odot$? 

Recall that for the Isothermal sphere :
$\rho(r) = \frac{V_{LSR}^2}{4\pi G r^2}$

Where $G$ = 4.4988e-6 kpc$^3$/Gyr$^2$/M$_\odot$

What about at 260 kpc (in units of 10$^{12}$ M$_\odot$) ? 

In [7]:
from astropy.constants import G
G.to(u.kpc**3 / u.Gyr**2 / u.M_sun)

<Quantity 4.49850215e-06 kpc3 / (Gyr2 solMass)>

We have a period $P$ and radius $R_0$ for the Sun's orbit and can take the mass distribution to be spherically symmetric, so equivalent to a point mass at the center. Then from Kepler III the mass enclosed within the orbit is

$$ M(R_0) = \frac{4 \pi^2}{G} \frac{R_0^3}{P^2} $$

In [8]:
M_R_0 = 4 * np.pi**2 / G * R_0[1]**3 / gravity_period**2
M_R_0.to(u.M_sun)

<Quantity 1.05038026e+11 solMass>

In the units asked for, $M(R_0) \approx 10.5 \times (10^{10} M_{\odot})$

In general, $M(R) = \int_0^R 4 \pi r^2 \rho(r) dr$.

For the isothermal sphere, $\rho(r) \propto r^{-2}$. Radius $r$ thus cancels from the term inside the integral and we have a linear relationship $M(r) \propto r$.

Using the result for the Sun's radius $R_0$ calculated above, the mass enclosed at arbitrary radius is 

$$ M(r) = \frac{r}{R_0} M(R_0) $$

At 260 kpc:

In [9]:
M_260 = 260*u.kpc / R_0[1] * M_R_0
M_260.to(u.M_sun)

<Quantity 3.33943344e+12 solMass>

### c) 

The Leo I satellite is one of the fastest moving satellite galaxies we know. 


It is moving with 3D velocity of magnitude: Vtot = 196 km/s at a distance of 260 kpc (Sohn 2013 ApJ 768)

If we assume that Leo I is moving at the escape speed:

$v_{esc}^2 = 2|\Phi| = 2 \int G \frac{\rho(r)}{r}dV $ 

and assuming the Milky Way is well modeled by a Hernquist Sphere with a scale radius of $a$= 30 kpc, what is the minimum mass of the Milky Way (in units of $10^{12}$ M$_\odot$) ?  

How does this compare to estimates of the mass assuming the Isothermal Sphere model at 260 kpc (from your answer above)

In [10]:
a = 30 * u.kpc
R = 260 * u.kpc
v_esc = 196 * u.km/u.s

For the Herquist Sphere, the gravitational potential is 
$$ \Phi(r) = \frac{-G M}{(r+a)} $$
and this has a simple relation to the escape velocity
$$ v_{esc}^2 = 2 |\Phi| = 2 \frac{G M}{(r+a)} \qquad \Rightarrow \qquad M = \frac{v_{esc}^2 (r+a)}{2G} $$

At the radius of Leo I:

In [11]:
M = v_esc**2*(R+a)/(2*G)
M.to(u.M_sun)

<Quantity 1.29514698e+12 solMass>

This is about a third of the Isothermal Sphere estimate.